In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import numpy.random as rnd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%precision 3

1\. **Radioactive decay chain**

${\rm Tl}^{208}$ decays to ${\rm Pb}^{208}$ with a half-lieve of 3.052 minutes. Suppose to start with a sample of 1000 Thallium atoms and 0 of Lead atoms.

* Take steps in time of 1 second and at each time-step decide whether each Tl atom has decayed or not, accordingly to the probability $p(t)=1-2^{-t/\tau}$. Subtract the total number of Tl atoms that decayed at each step from the Tl sample and add them to the Lead one. Plot the evolution of the two sets as a function of time  
* Repeat the exercise by means of the inverse transform method: draw 1000 random numbers from the non-uniform probability distribution $p(t)=2^{-t/\tau}\frac{\ln 2}{\tau}$ to represent the times of decay of the 1000 Tl atoms. Make a plot showing the number of atoms that have not decayed as a function of time

##### Answer 1)

In [ ]:
tau = 3.052*60 
TI_num = 1000.0
Pb_num = 0.0

# define the prob. distribution function:
def prob(t):
    a = 1 - (2**(-t/tau)) 
    return a
t = 1 #TIME_STEP 
pb_list = []
ti_list = []
iteration = 200
epsilon = 1e-3


for i in range(iteration,t):
    for i in range ():
        a = rnd.rand.uniform(1,2)
        if a <= p(i):
        N[i] = 1000
        M[i] = N - 1000
    Pb_num = TI_num*prob(j) + Pb_num
    TI_num = TI_num - TI_num*prob(j)
    pb_list.append(Pb_num)
    ti_list.append(TI_num)
#print('\nNumber of Thallium atoms\n',ti_list,'\n\nNumber of Lead atoms\n',pb_list)  

# Plot the results
lin = np.linspace(1,1000,iteration)
plt.plot(lin,pb_list,lin,ti_list)
plt.grid()
plt.xlabel('Time-Steps')
plt.ylabel('Number of Atoms')
labels = ['Lead Atoms','Thiliam Atoms']
plt.legend(labels)
plt.show()


2\. **Rutherford Scattering**

The scattering angle $\theta$ of $\alpha$ particles hitting a positively charged nucleus of a Gold atom ($Z=79$) follows the rule:

$$
\tan{\frac{1}{2} \theta} = \frac{Z e^2} {2\pi \epsilon_0 E b}
$$

where $E=7.7$ MeV and $b$ beam is the impact parameter. The beam is represented by a 2D gaussian distribution with $\sigma=a_0/100$ for both coordinates ($a_0$ being the Bohr radius). Assume 1 million $\alpha$ particles are shot on the gold atom.

Computing the fraction of particles that "bounce back",i.e. those particle whose scattering angle is greater than $\pi/2$ (which set a condition on the impact parameter $b$)

##### Answer 2)

3\. **Monte Carlo integration: hit/miss vs mean value method**

Consider the function 

$$f(x) =\sin^2{\frac{1}{x(2-x)}}$$

* Compute the integral of $f(x)$ between 0 and 2 with the hit/miss method. Evaluate the error of your estimate
* Repeat the integral with the mean value method. Evaluate the error and compare it with the previous one

In [ ]:
def f(x):
    return (np.sin(1/(x*(2-x))))**2

x=np.linspace(0.001,1.999,1000)
plt.plot(x,f(x),'r-')
plt.grid()
plt.show()

## hit/miss method
## x = a + r1(b-a) , y = r2*f_max ===> N
N=100000
count=0
for i in range(N):
    x=2*np.random.random()
    y=np.random.random()
    if y<f(x): count+=1
I=2*count/N
print("Monte Carlo Integration = ",I)
print("Error for Monte Carlo Method = ",1/np.sqrt(N)) ### it could be proved due to literature which sigma_M = sigma / sqrt(N)

## Mean Value Method
x2 = np.linspace(0.00001,1.99999,100000)
y2 = np.sum(f(x2))
I2 = 2*y2/N
print("\nMean Value Method = ",I2)
print("Error for Mean value Method = ",1/np.sqrt(N))


4\. **Monte Carlo integration in high dimension**

* Start of by computing the area of a circle of unit radius, by integrating the function 

$$
f(x,y)=
\left\{
\begin{array}{ll}
      1 & x^2+y^2\le 1 \\
      0 & {\rm elsewhere}
\end{array} 
\right.
$$

* Generalize the result for a 10D sphere



##### Answer 4)

In [ ]:
def f(x,y):
    return (x**2+y**2)

N=100000
count1=0
for i in range(N):
    x=np.random.random()
    y=np.random.random()
    if f(x,y)<=1: count1+=1
I=4*count1/N
print("Area of the Circle = ",I)


## Generalize for 10D Sphere
def fd(d):
    a,b = 0,0
    for i in range(d+1):
        a = np.random.random()
        a,b = b,a**2+b
    return (a)

## dummy Method
def k(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10):
    return (x1**2+x2**2+x3**2+x4**2+x5**2+x6**2+x7**2+x8**2+x9**2+x10**2)

N=100000
count2=0
for i in range(N):
    if fd(d=10)<=1: count2+=1
I1=(2**10)*count2/N
print("Area of 10D Sphere = ",I1)

## Dummy Method to check
N=100000
count3=0
for i in range(N):
    x1=np.random.random()
    x2=np.random.random()
    x3=np.random.random()
    x4=np.random.random()
    x5=np.random.random()
    x6=np.random.random()
    x7=np.random.random()
    x8=np.random.random()
    x9=np.random.random()
    x10=np.random.random()
    if k(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10)<=1: count3+=1
I2=(2**10)*count3/N
print("Area of 10D Sphere = ",I2)

5\. **Monte Carlo integration with importance sampling** 

Calculate the value of the integral:

$$
I=\int_0^1 \frac{x^{-1/2}}{e^x+1} dx
$$

using the importance sampling method with $w(x)=1/\sqrt{x}$. You should get a result about 0.84

##### Answer 5)

In [ ]:
def g(x):
    return (x**(-(1/2))/(np.exp(x)+1))

def w(x):
    return (1/np.sqrt(x))

N=100000
count=0
for i in range(N):
    x=np.random.random()
    y=w(0.000001)*np.random.random()
    if y<w(x): count+=1
I_w=w(0.000001)*count/N

x2 = np.linspace(0.00001,0.99999,100000)
I_g = np.sum(g(x2)/w(x2))
I_importance = I_g*I_w*(1/N)
print("result for Importance sampling method = ",I_importance)
    
